# Link based ranking

**Preliminaries:** If you want to normalize a vector to L1-norm or L2-norm, use:

In [183]:
from __future__ import print_function, division
import numpy as np
from tqdm import tqdm

pr = np.array([1,2,3])
print("L1-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,1)))
print("L2-norm of {0} is {1}".format(pr, pr / np.linalg.norm(pr,2)))

L1-norm of [1 2 3] is [0.16666667 0.33333333 0.5       ]
L2-norm of [1 2 3] is [0.26726124 0.53452248 0.80178373]


## Page Rank (Eigen-vector method)
Consider a tiny Web with three pages A, B and C with no inlinks,
and with initial PageRank = 1. Initially, none of the pages link to
any other pages and none link to them. 
Answer the following questions, and calculate the PageRank for
each question.

1. Link page A to page B.
2. Link all pages to each other.
3. Link page A to both B and C, and link pages B and C to A.
4. Use the previous links and add a link from page C to page B.

**Hints:**
We are using the theoretical PageRank computation (without source of rank). See slide "Transition Matrix for Random Walker" in the lecture note. Columns of link matrix are from-vertex, rows of link matrix are to-vertex. We take the eigenvector with the largest eigenvalue.
We only care about final ranking of the probability vector. You can choose the normalization (or not) of your choice).

In [184]:
def create_Rmatrix(L):
#     R = np.multiply(L, 1 / np.sum(L,axis=0)) # Use this matrix multiplication for faster running time if no column is zero
    X = np.sum(L,axis=0)
    n_nodes = L.shape[0]
    R = np.zeros((n_nodes, n_nodes))
    for i in tqdm(range(L.shape[0])):
        for j in range(L.shape[1]):
            R[i,j] = L[i,j] / X[0,j] if X[0,j] != 0 else 0
            
#     R = np.multiply(L,R)
    return R

# Implement your code here
def pagerank_eigen(L):
#   Construct transition probability matrix from L
    # if zeros in the columns are included
    R = create_Rmatrix(L)
    #R = L / np.sum(L, axis=0) # no column would need to be zero!
    
    
#     Compute eigen-vectors and eigen-values of R
    eigenvalues, eigenvectors = np.linalg.eig(R)
#     Take the eigen-vector with maximum eigven-value
    p = eigenvectors[:,np.argmax(np.absolute(eigenvalues))]
    p = np.absolute(p)
    p = p/np.sum(p)
    return (R,p)

In [185]:
# Test with the question, e.g.
L = np.matrix([
    [0,0,1], 
    [1,0,0], 
    [1,1,0]
])
R,p = pagerank_eigen(L)
print("L={0}\nR={1}\np={2}".format(L,R,p))

100%|██████████████████████████████████████████| 3/3 [00:00<00:00, 16049.63it/s]

L=[[0 0 1]
 [1 0 0]
 [1 1 0]]
R=[[0.  0.  1. ]
 [0.5 0.  0. ]
 [0.5 1.  0. ]]
p=[0.4 0.2 0.4]


## Page Rank (Iterative method)

The eigen-vector method has some numerical issues (when computing eigen-vector) and not scalable with large datasets.

We will apply the iterative method in the slide "Practical Computation of PageRank" of the lecture.

Dataset for practice: https://snap.stanford.edu/data/ca-GrQc.html. It is available within the same folder of this github.

In [186]:
def pagerank_iterative(L):
    #R = L / np.sum(L, axis=0)
    R = create_Rmatrix(L)
    N = L.shape(0)
    e = np.ones(shape=(N,1))
    q = 0.9

    p = e
    delta = 1
    epsilon = 0.001
    i = 0
    while delta > epsilon:
        p_prev = p
        p = ((R*q)@p)
        p = p + ((1-q)/N)*e
        delta = np.linalg.norm(p-p_prev,1)
        i += 1
        print(p)

    print("Converged after {0} iterations. Ranking vector: p={1}".format(i, p[:,0]))
    return R,p

### Test with the dataset


In [187]:
# Construct link matrix from file
f = open("ca-GrQc.txt")
link_list = []
max_value = -1
for line in f.readlines():
    if line[0] == "#":
        continue
    arr = [int(x) for x in line.split()]
    if arr[0] > max_value:
        max_value = arr[0]
    if arr[1] > max_value:
        max_value = arr[1]

    link_list.append(arr)

link_matrix = np.zeros((max_value+1, max_value+1))
for from_node, to_node in link_list:
    link_matrix[to_node, from_node] = 1


L = np.matrix(link_matrix)
R = create_Rmatrix(L)

100%|█████████████████████████████████████| 26197/26197 [11:01<00:00, 39.62it/s]


In [188]:
# Run PageRank
R, p = pagerank_iterative(L)
print("Ranking vector: p={0}".format(p[:,0]))

100%|█████████████████████████████████████| 26197/26197 [10:53<00:00, 40.08it/s]


TypeError: 'tuple' object is not callable

## Ranking Methodology (Hard)

1. Give a directed graph, as small as possible, satisfying all the properties mentioned below:

    1. There exists a path from node i to node j for all nodes i,j in the directed
graph. Recall, with this property the jump to an arbitrary node in PageRank
is not required, so that you can set q = 1 (refer lecture slides).

    2. HITS authority ranking and PageRank ranking of the graph nodes are different.

2. Give intuition/methodology on how you constructed such a directed graph with
the properties described in (a).

3. Are there specific graph structures with arbitrarily large instances where PageRank
ranking and HITS authority ranking are the same?

## Question 4 - Hub and Authority

### a)

Let the adjacency matrix for a graph of four vertices ($n_1$ to $n_4$) be
as follows:

$
A =
  \begin{bmatrix}
	0 & 1 & 1 & 1  \\
	0 & 0 & 1 & 1 \\
	1 & 0 & 0 & 1 \\
	0 & 0 & 0 & 1 \\
  \end{bmatrix}
$

Calculate the authority and hub scores for this graph using the
HITS algorithm with k = 6, and identify the best authority and
hub nodes. (A non-zero element $A_{ij}$ indicates an edge from $i$ to $j$.)

### b)
Apply the HITS algorithm to the dataset: https://snap.stanford.edu/data/ca-GrQc.html

**Hint:** We follow the slide "HITS algorithm" in the lecture. **Denote $x$ as authority vector and $y$ as hub vector**. You can use matrix multiplication for the update steps in the slide "Convergence of HITS". Note that rows of adjacency matrix is from-vertex and columns of adjacency matrix is to-vertex.

In [198]:
# You can implement your code following this template.
def hits_iterative(A, k=10):
    N = A.shape[0]

    v0, u0 = 1 / (N*N) * np.ones(N), 1 / (N*N) * np.ones(N) 

    vprev, uprev = v0, u0
    
    # For advanced exercise: define a convergence condition instead of k iterations
    for l in range(0,k):
        v = A.T@uprev
        u = A@vprev
        vprev = v/np.linalg.norm(v,2)
        uprev = u/np.linalg.norm(u,2)
        
    return vprev, uprev

In [203]:
A = np.array([
    [0,1,1,1], 
    [0,0,1,1], 
    [1,0,0,1],
    [0,0,0,1]
])
hits_iterative(A,k=3)

(array([0.18543974, 0.25961563, 0.48214331, 0.81593484]),
 array([0.65926851, 0.53565567, 0.41204282, 0.32963426]))